In [ ]:
# Install Dependencies
!pip install -q transformer-lens datasets tqdm accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00


In [ ]:
# Import Libraries
import torch
from transformer_lens import HookedTransformer
import numpy as np
from IPython.display import HTML, display
import pandas as pd
from typing import List, Tuple

In [ ]:
# Get Max Neuron Activation and Visualize

model = HookedTransformer.from_pretrained("gelu-1l", center_unembed=True)
tok = model.tokenizer
model.eval()
LAYER = 0
NEURON = 2029

activations = {}
def save_mlp_post(act, hook):
    activations["mlp_post"] = act.detach().cpu()

model.add_hook(f"blocks.{LAYER}.mlp.hook_post", save_mlp_post)

def get_max_neuron_activation_with_context(text: str, window: int = 3):

    inputs = tok(text, return_tensors="pt")
    _ = model(inputs.input_ids)

    acts = activations["mlp_post"][0, :, NEURON]
    max_val, max_idx = acts.max(dim=0)
    max_idx = max_idx.item()
    max_val = max_val.item()

    seq_len = inputs.input_ids.size(1)
    start = max(0, max_idx - window)
    end   = min(seq_len, max_idx + window + 1)

    context_ids = inputs.input_ids[0, start:end].tolist()
    context_str = tok.decode(context_ids)

    token_id = inputs.input_ids[0, max_idx].item()
    token_str = tok.decode([token_id])

    return {
        "token": token_str,
        "position": max_idx,
        "activation": max_val,
        "context": context_str
    }

def visualize_activations(text: str):

    inputs = tok(text, return_tensors="pt")
    _ = model(inputs.input_ids)

    acts = activations["mlp_post"][0, :, NEURON]

    min_a, max_a = acts.min(), acts.max()
    norm_acts = (acts - min_a) / (max_a - min_a + 1e-10)

    html_chunks = []
    for idx, (token_id, score) in enumerate(zip(inputs.input_ids[0], norm_acts)):
        token_str = tok.decode([token_id.item()])
        color = f"rgba(225,76,76,{score:.2f})"
        html_chunks.append(
            f"<span style='background-color:{color}; padding:2px; border-radius:3px;'>{token_str}</span>"
        )

    html = " ".join(html_chunks)
    display(HTML(f"<div style='line-height:1.6; font-size:1.1em'>{html}</div>"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

model_final.pth:   0%|          | 0.00/213M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.04M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Loaded pretrained model gelu-1l into HookedTransformer


In [ ]:
# Run example text and visualize

example = '''
The kings and the queens and the princes.
'''
visualize_activations(example)
res = get_max_neuron_activation_with_context(example)
print(f"Max‐activating token: {res['token']} (position {res['position']})")
print(f"Activation value: {res['activation']:.4f}")
print(f"Context window: \"{res['context']}\"")

Max‐activating token:  and (position 3)
Activation value: 2.3935
Context window: "
The kings and the queens"


In [ ]:
# Print neuron 2029's top ten most boosted tokens and top ten least boosted tokens

mlp    = model.blocks[LAYER].mlp
W_out  = mlp.W_out.detach().cpu()
W_U    = model.unembed.W_U.detach().cpu()

print("W_out shape:", W_out.shape)
print("W_U shape:  ", W_U.shape)
d_mlp, d_model = W_out.shape

proj_vec = W_out[NEURON]

logit_weights = proj_vec @ W_U

topk_pos = torch.topk(logit_weights, k=10)
topk_neg = torch.topk(logit_weights, k=10, largest=False)

print("\nTop 10 tokens BOOSTED by neuron", NEURON)
for idx, score in zip(topk_pos.indices.tolist(), topk_pos.values.tolist()):
    token = model.tokenizer.decode([idx]).strip()
    print(f"  {token!r}: {score:.2f}")

print("\nTop 10 tokens SUPPRESSED by neuron", NEURON)
for idx, score in zip(topk_neg.indices.tolist(), topk_neg.values.tolist()):
    token = model.tokenizer.decode([idx]).strip()
    print(f"  {token!r}: {score:.2f}")

W_out shape: torch.Size([2048, 512])
W_U shape:   torch.Size([512, 48262])

Top 10 tokens BOOSTED by neuron 2029
  ']):': 1.08
  ')):': 0.94
  '': 0.86
  'affecting': 0.85
  ')</': 0.85
  'prompted': 0.85
  '>]': 0.85
  ')--': 0.84
  'occurs': 0.83
  'costing': 0.83

Top 10 tokens SUPPRESSED by neuron 2029
  'ancer': -0.95
  'ogy': -0.94
  'abad': -0.94
  'osed': -0.93
  'proof': -0.91
  'enches': -0.90
  'ulous': -0.90
  'arers': -0.90
  'aset': -0.87
  'inned': -0.86


In [ ]:
# Print top ten co-activiating neurons for each feature prompt

def top_coactivating_neurons(text: str, top_k: int = 10):
    toks = tok(text, return_tensors="pt")
    _ = model(toks.input_ids)

    acts = activations["mlp_post"][0]

    peak, _ = acts.max(dim=0)
    vals, idxs = torch.topk(peak, k=top_k)

    print(f"\nTop {top_k} co-activating neurons for “{text[:30]}…”:")
    for i, v in zip(idxs.tolist(), vals.tolist()):
        print(f"  Neuron {i:4d}  peak act = {v:.3f}")

top_coactivating_neurons("the kings and three camels")
top_coactivating_neurons("for i in range(10): print(i)")


Top 10 co-activating neurons for “the kings and three camels…”:
  Neuron 2019  peak act = 6.202
  Neuron  411  peak act = 6.052
  Neuron  109  peak act = 5.842
  Neuron 1208  peak act = 5.102
  Neuron 1651  peak act = 4.139
  Neuron 1590  peak act = 3.592
  Neuron 1582  peak act = 3.171
  Neuron   64  peak act = 3.163
  Neuron 1476  peak act = 3.026
  Neuron  176  peak act = 2.742

Top 10 co-activating neurons for “for i in range(10): print(i)…”:
  Neuron 1712  peak act = 4.840
  Neuron  474  peak act = 4.208
  Neuron 1910  peak act = 3.711
  Neuron  217  peak act = 3.699
  Neuron   96  peak act = 3.000
  Neuron  526  peak act = 2.834
  Neuron 1446  peak act = 2.729
  Neuron    9  peak act = 2.659
  Neuron  836  peak act = 2.590
  Neuron 1918  peak act = 2.525


In [ ]:
# Print the top-k logits at the final position and their tokens with and without neuron 2029 ablated.

test_prompt = "The king and queen are great."
TOP_K       = 10

def reset():
    model.reset_hooks()

def print_topk_logits(logits, k=10):

    last_logits = logits[0, -1]
    topk = torch.topk(last_logits, k)

    print(f"Top {k} logits:")

    for idx, score in zip(topk.indices.tolist(), topk.values.tolist()):
        token = tok.decode([idx]).strip()
        print(f"  {token!r}: {score:.4f}")

    print()

reset()
logits_base, _ = model.run_with_cache(test_prompt)
print("=== Baseline ===")
print_topk_logits(logits_base, TOP_K)

reset()
def ablate_fn(acts, hook):
    acts[..., NEURON] = 0.0
    return acts

model.add_hook(f"blocks.{LAYER}.mlp.hook_post", ablate_fn)
logits_abl, _ = model.run_with_cache(test_prompt)
print("=== Ablated ===")
print_topk_logits(logits_abl, TOP_K)

=== Baseline ===
Top 10 logits:
  '': 15.6600
  'The': 15.2588
  'I': 14.4898
  'They': 14.1936
  'It': 14.1037
  'We': 13.6630
  'There': 13.6133
  'You': 13.6077
  'This': 13.3798
  'And': 13.3797

=== Ablated ===
Top 10 logits:
  '': 15.6788
  'The': 15.2415
  'I': 14.5089
  'They': 14.1660
  'It': 14.0907
  'We': 13.6476
  'There': 13.6134
  'You': 13.6119
  'And': 13.3872
  'This': 13.3823



In [ ]:
# Script for identifying and ablating cancellers as well as baseline comparison

DEVICE        = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME    = "gelu-1l"
LAYER         = 0
NEURON_ID     = 2029
PROMPT_A      = "the king and"
PROMPT_B      = "for i in range"
TOP_CANCELERS = 5
TOP_K_PRINT   = 10

model = HookedTransformer.from_pretrained(MODEL_NAME, center_unembed=True).to(DEVICE).eval()
tok   = model.tokenizer

W_out = model.blocks[LAYER].mlp.W_out.detach()
W_U   = model.unembed.W_U.detach()
d_mlp, d_model = W_out.shape

def run_with_cache(prompt: str):
    return model.run_with_cache(
        prompt,
        names_filter=[f"blocks.{LAYER}.mlp.hook_post"],
        device=DEVICE,
    )

def resid_split(prompt: str) -> Tuple[torch.Tensor, torch.Tensor]:
    _, cache = run_with_cache(prompt)
    acts = cache[f"blocks.{LAYER}.mlp.hook_post"][0][-1]
    v_amb = acts[NEURON_ID] * W_out[NEURON_ID]
    rest  = (acts @ W_out) - v_amb
    return v_amb, rest

def cosine(a, b): return torch.dot(a, b).item() / (a.norm() * b.norm() + 1e-9)

def print_topk_logits(vec_or_logits: torch.Tensor, k=10, note=""):

    if vec_or_logits.shape[0] == W_U.shape[1]:
        logits = vec_or_logits
    else:
        logits = vec_or_logits @ W_U
    topk = torch.topk(logits, k)
    print(f"\nTop {k} tokens for {note}:")
    for idx, score in zip(topk.indices.tolist(), topk.values.tolist()):
        token = tok.decode([idx]).strip()
        print(f"  {token!r:<10} {score:7.2f}")

def ablate_neurons(prompt: str, neuron_ids: List[int]) -> torch.Tensor:
    def hook_fn(acts, hook):
        acts[..., neuron_ids] = 0.0
        return acts
    return model.run_with_hooks(
        prompt,
        return_type="logits",
        fwd_hooks=[(f"blocks.{LAYER}.mlp.hook_post", hook_fn)],
    )

# 1. Identify cancellation direction
vA, restA = resid_split(PROMPT_A)
vB, restB = resid_split(PROMPT_B)

print("\n=== COSINE(v_ambiguous, rest_of_stream) ===")
print(f"Prompt A ({PROMPT_A[:15]}…): {cosine(vA, restA):+.3f}")
print(f"Prompt B ({PROMPT_B[:15]}…): {cosine(vB, restB):+.3f}")

# 2. Identify Cancellers for Prompt A
_, cacheA = run_with_cache(PROMPT_A)
actsA = cacheA[f"blocks.{LAYER}.mlp.hook_post"][0][-1]
contrA = actsA[:, None] * W_out
align_scores = torch.mv(contrA, vA) / (vA.norm() + 1e-9)

cancelers = align_scores.topk(TOP_CANCELERS, largest=False).indices.tolist()
helpers   = align_scores.topk(TOP_CANCELERS, largest=True ).indices.tolist()

print("\nTop canceling neurons (Prompt A):", cancelers)
print("Top reinforcing neurons (Prompt A):", helpers)

# 3. Causal test with baseline and ablated top-k
def show_logit_change(prompt: str, neuron_group: List[int], label: str):
    base_logits = model(prompt)
    ablated     = ablate_neurons(prompt, neuron_group)

    last_base = base_logits[0, -1]
    last_ab   = ablated[0, -1]

    target = "]):"
    tgt_id = tok.encode(target)[0]
    margin_base = last_base[tgt_id] - last_base.max()
    margin_ab   = last_ab[tgt_id]  - last_ab.max()

    print(f"\n→ Patch test on prompt: {prompt!r}")
    print(f"  Target token: '{target}'")
    print(f"  Margin baseline: {margin_base:+.3f}   after ablation ({label}): {margin_ab:+.3f}")

    print_topk_logits(last_base, TOP_K_PRINT, note="baseline")
    print_topk_logits(last_ab,   TOP_K_PRINT, note=f"ablated {label}")

show_logit_change(PROMPT_A, cancelers, "cancelers")
show_logit_change(PROMPT_B, cancelers, "cancelers")

# 4. Display raw vector top tokens (as before)
print_topk_logits(W_out[NEURON_ID], 20, note="raw W_out[2029]")


Loaded pretrained model gelu-1l into HookedTransformer
Moving model to device:  cuda

=== COSINE(v_ambiguous, rest_of_stream) ===
Prompt A (the king and…): +0.294
Prompt B (for i in range…): +0.270

Top canceling neurons (Prompt A): [1109, 808, 1303, 33, 537]
Top reinforcing neurons (Prompt A): [2029, 265, 1301, 1061, 200]

→ Patch test on prompt: 'the king and'
  Target token: ']):'
  Margin baseline: -17.900   after ablation (cancelers): -14.319

Top 10 tokens for baseline:
  'queen'      14.47
  'the'        13.83
  'his'        13.32
  'king'       12.78
  'a'          12.22
  'he'         11.93
  'her'        11.48
  'all'        11.37
  'it'         11.32
  'their'      11.32

Top 10 tokens for ablated cancelers:
  'the'        13.01
  'queen'      12.70
  'a'          12.35
  'his'        12.17
  'king'       11.67
  'he'         11.10
  'is'         10.93
  'in'         10.92
  ','          10.79
  'was'        10.75

→ Patch test on prompt: 'for i in range'
  Target token: '])

In [ ]:
# Scipt for showing that *small, different set of neighbours cancels (or reinforces) neuron 2029 in two distinct contexts.

DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
MODEL   = "gelu-1l"
LAYER   = 0
NEURON  = 2029
K       = 5
PROMPT_A = "the king and"
PROMPT_B = "for i in range"

model = HookedTransformer.from_pretrained(MODEL, center_unembed=True).to(DEVICE).eval()
tok   = model.tokenizer

W_out = model.blocks[LAYER].mlp.W_out.detach()
W_U   = model.unembed.W_U.detach()

def resid_and_acts(prompt: str):

    logits, cache = model.run_with_cache(
        prompt,
        names_filter=[f"blocks.{LAYER}.mlp.hook_post"],
        device=DEVICE,
    )
    acts = cache[f"blocks.{LAYER}.mlp.hook_post"][0][-1]
    resid = acts @ W_out
    return acts, resid

def top_cancelers(acts, v_amb, k: int) -> List[int]:

    contr = acts[:, None] * W_out
    scores = torch.mv(contr, v_amb) / (v_amb.norm()+1e-9)
    return scores.topk(k, largest=False).indices.tolist()

def logit_margin(logits, target_id):
    last = logits[0, -1]
    return (last[target_id] - last.max()).item()

def ablate(prompt: str, neurons: List[int]):
    def hook_fn(acts, hook): acts[..., neurons] = 0.0; return acts
    return model.run_with_hooks(
        prompt,
        return_type="logits",
        fwd_hooks=[(f"blocks.{LAYER}.mlp.hook_post", hook_fn)]
    )

# 1.  Get ambiguous vector once (same direction any prompt)
acts_A, _ = resid_and_acts(PROMPT_A)
v_amb = acts_A[NEURON] * W_out[NEURON]

# 2.  Build canceler sets for each prompt
acts_A, _ = resid_and_acts(PROMPT_A)
acts_B, _ = resid_and_acts(PROMPT_B)

cancel_A = top_cancelers(acts_A, v_amb, K)
cancel_B = top_cancelers(acts_B, v_amb, K)


print(f"\nCancelers for Prompt A: {cancel_A}")
print(f"Cancelers for Prompt B: {cancel_B}")
print(f"Union size = {len(set(cancel_A) | set(cancel_B))}  (should be small)")

# 3.  Measure margin changes
def test(prompt:str, cancelers:List[int], label:str):
    target = "]):"
    tgt_id = tok.encode(target)[0]
    base_logits = model(prompt)
    abl_logits  = ablate(prompt, cancelers)
    m_base = logit_margin(base_logits, tgt_id)
    m_abl  = logit_margin(abl_logits,  tgt_id)
    print(f"\nPrompt: {prompt!r}")
    print(f"Using cancelers from {label}")
    print(f"  logit margin baseline  : {m_base:+.3f}")
    print(f"  after ablation ({label}): {m_abl:+.3f}   Δ = {m_abl-m_base:+.3f}")

print("\n─── In‑context ablations ───────────────────────────")
test(PROMPT_A, cancel_A, "the king and")
test(PROMPT_B, cancel_B, "for i in range")

print("\n─── Cross‑context ablations ─────────────────────────")
test(PROMPT_A, cancel_B, "for i in range")
test(PROMPT_B, cancel_A, "the king and")


Loaded pretrained model gelu-1l into HookedTransformer
Moving model to device:  cuda

Cancelers for Prompt A: [1109, 808, 1303, 33, 537]
Cancelers for Prompt B: [1908, 697, 515, 82, 143]
Union size = 10  (should be small)

─── In‑context ablations ───────────────────────────

Prompt: 'the king and'
Using cancelers from the king and
  logit margin baseline  : -17.900
  after ablation (the king and): -14.319   Δ = +3.581

Prompt: 'for i in range'
Using cancelers from for i in range
  logit margin baseline  : -10.809
  after ablation (for i in range): -6.868   Δ = +3.941

─── Cross‑context ablations ─────────────────────────

Prompt: 'the king and'
Using cancelers from for i in range
  logit margin baseline  : -17.900
  after ablation (for i in range): -17.740   Δ = +0.160

Prompt: 'for i in range'
Using cancelers from the king and
  logit margin baseline  : -10.809
  after ablation (the king and): -10.952   Δ = -0.143
